# Gridded EPA Methane Inventory
## Category: 1B1a Abandoned Coal Mines

***
#### Authors: 
Joannes D. Maasakkers, Erin E. McDuffie
#### Date Last Updated: 
see Step 0
#### Notebook Purpose: 
This Notebook calculates and saves annual gridded (0.1°x0.1°) methane emission fluxes (molec./cm2/s) from abandoned coal mines in the CONUS region between 2012-2018.    
#### Summary & Notes:
EPA GHGI abandoned underground coal mine emissions are read in at the national level from the GHGI (from GHGI workbook). Net liberated and recovered & used emissions are grouped into a single emissions group. National emissions are allocated to relative mine-level emission estimates. The complete list of abandoned mines is from 2021 GHGI workbook (largely from the MSHA database), which includes active emission estimates (at the time of abandonment) and MSHA IDs. Mine emissions are calculated using emission decay curves (also used in the GHGI) as a function of mine type, basin, and the number of years since closure. National emissions are then mapped to a 0.01°x0.01° grid using latitude and longitude values from the MSHA master mine database (matched based on ID, mine name, state, and county). For mines on the master list whose location could not be found, emissions are allocated across the county on record for that mine.  All emissions are re-gridded to a 0.1°x0.1° grid. Emissions are converted to emission flux. Annual emission fluxes (molec./cm2/s) are written to final netCDFs in the ‘/code/Final_Gridded_Data/’ folder.
***

-------
## Step 0. Set-Up Notebook Modules, Functions, and Local Parameters and Constants
-------

In [ ]:
#Confirm working directory
import os
import time
modtime = os.path.getmtime('./v2_1B1a_Coal_Abandoned.ipynb')
modificationTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(modtime))
print("This file was last modified on: ", modificationTime)
print('')
print("The directory we are working in is {}" .format(os.getcwd()))

In [ ]:
## Include plots within notebook
%matplotlib inline

In [ ]:
# Import base modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import datetime
from copy import copy

# Import additional modules
# Load plotting package Basemap 
from mpl_toolkits.basemap import Basemap

# Load netCDF (for manipulating netCDF file types)
from netCDF4 import Dataset

# Set up ticker
import matplotlib.ticker as ticker

#add path for the global function module (file)
import sys
module_path = os.path.abspath(os.path.join('../Global_Functions/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load Tabula (for reading tables from PDFs)
import tabula as tb   
    
# Load user-defined global functions (modules)
import data_load_functions as data_load_fn
import data_functions as data_fn
import data_IO_functions as data_IO_fn
import data_plot_functions as data_plot_fn

In [ ]:
#INPUT Files
# Assign global file names
global_filenames = data_load_fn.load_global_file_names()
State_ANSI_inputfile = global_filenames[0]
County_ANSI_inputfile = global_filenames[1]
pop_map_inputfile = global_filenames[2]
Grid_area01_inputfile = global_filenames[3]
Grid_area001_inputfile = global_filenames[4]
Grid_state001_ansi_inputfile = global_filenames[5]
Grid_county001_ansi_inputfile = global_filenames[6]
globalinputlocation = global_filenames[0][0:20]
print(globalinputlocation)

# Specify names of inputs files used in this notebook
#EPA Data
EPA_abdcoal_inputfile = '../Global_InputData/GHGI/Ch3_Energy/AbandonedCoalMines1990-2018_02.20.20.xlsm'
EPA_abdcoal_2019_inputfile = '../Global_InputData/GHGI/Ch3_Energy/AbandonedCoalMines1990-2019_erg_03-04-2021.xlsm'

#Proxy Data file
AbdCoal_Mapping_inputfile = "./InputData/AbandonedCoal_ProxyMapping.xlsx"


#Activity Data
Mine_loc_inputfile = "../Global_InputData/MSHA/Mines.txt"
AbdMine_list_inputfile = "./InputData/amm_opportunities_list.csv"


#OUTPUT FILES
gridded_outputfile = '../Final_Gridded_Data/EPA_v2_1B1a_Abandoned_Coal.nc'
netCDF_description = 'Gridded EPA Inventory - Abandoned Underground Coal Mine Emissions - IPCC Source Category 1B1a'
title_str = "EPA methane emissions from abandoned coal mines"
title_diff_str = "Emissions from abandoned coal mines difference: 2018-2012"

#output gridded proxy data
grid_emi_outputfile = '../Final_Gridded_Data/Extension/v2_input_data/Abandoned_Coal_Grid_Emi.nc'

In [ ]:
# Define local variables
start_year = 2012  #First year in emission timeseries
end_year = 2018    #Last year in emission timeseries
year_range = [*range(start_year, end_year+1,1)] #List of emission years
year_range_str=[str(i) for i in year_range]
num_years = len(year_range)

# Define constants
Avogadro   = 6.02214129 * 10**(23)  #molecules/mol
Molarch4   = 16.04                  #g/mol
Res01      = 0.1                    # degrees
Res_01     = 0.01
tg_scale   = 0.001                  #Tg scale number [New file allows for the exclusion of the territories] 

# Continental US Lat/Lon Limits (for netCDF files)
Lon_left = -130       #deg
Lon_right = -60       #deg
Lat_low  = 20         #deg
Lat_up  = 55          #deg
loc_dimensions = [Lat_low, Lat_up, Lon_left, Lon_right]

ilat_start = int((90+Lat_low)/Res01) #1100:1450 (continental US range)
ilat_end = int((90+Lat_up)/Res01)
ilon_start = abs(int((-180-Lon_left)/Res01)) #500:1200 (continental US range)
ilon_end = abs(int((-180-Lon_right)/Res01))

# Number of days in each month
month_day_leap  = [  31,  29,  31,  30,  31,  30,  31,  31,  30,  31,  30,  31]
month_day_nonleap = [  31,  28,  31,  30,  31,  30,  31,  31,  30,  31,  30,  31]

# Month arrays
month_range_str = ['January','February','March','April','May','June','July','August','September','October','November','December']
num_months = len(month_range_str)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
# Track run time
ct = datetime.datetime.now() 
it = ct.timestamp() 
print("current time:", ct) 

____
## Step 1. Load in State ANSI data and Area Maps
_____

In [ ]:
# State-level ANSI Data
#Read the state ANSI file array
State_ANSI, name_dict = data_load_fn.load_state_ansi(State_ANSI_inputfile)[0:2]
#QA: number of states
print('Read input file: '+ f"{State_ANSI_inputfile}")
print('Total "States" found: ' + '%.0f' % len(State_ANSI))
print(' ')

abbr_dict = State_ANSI.set_index('abbr')['ansi'].to_dict()
name_dict = State_ANSI.set_index('name')['ansi'].to_dict()

# 0.01 x0.01 degree Data
# State ANSI IDs and grid cell area (m2) maps
state_ANSI_map = data_load_fn.load_state_ansi_map(Grid_state001_ansi_inputfile)
area_map, lat001, lon001 = data_load_fn.load_area_map_001(Grid_area001_inputfile)

#County ANSI Data
#Includes State ANSI number, county ANSI number, county name, and country area (square miles)
County_ANSI = pd.read_csv(County_ANSI_inputfile,encoding='latin-1')
#QA: number of counties
print ('Read input file: ' + f"{County_ANSI_inputfile}")
print('Total "Counties" found (include PR): ' + '%.0f' % len(County_ANSI))
print(' ')
for icounty in np.arange(0,len(County_ANSI)):
    County_ANSI.loc[icounty,'Name'] = County_ANSI.loc[icounty,'Name'].lower().strip()
    
# 0.01 x0.01 degree Data
# State ANSI IDs and grid cell area (m2) maps
state_ANSI_map = data_load_fn.load_state_ansi_map(Grid_state001_ansi_inputfile)
state_ANSI_map = state_ANSI_map.astype('int32')
county_ANSI_map = data_load_fn.load_county_ansi_map(Grid_county001_ansi_inputfile)
county_ANSI_map = county_ANSI_map.astype('int32')
area_map, lat001, lon001 = data_load_fn.load_area_map_001(Grid_area001_inputfile)

# 0.1 x0.1 degree data
# grid cell area and state and county ANSI maps
area_map01, Lat01, Lon01 = data_load_fn.load_area_map_01(Grid_area01_inputfile)[0:3]
#Select relevant Continental 0.1 x0.1 domain
Lat_01 = Lat01[ilat_start:ilat_end]
Lon_01 = Lon01[ilon_start:ilon_end]
area_matrix_01 = data_fn.regrid001_to_01(area_map, Lat_01, Lon_01)
area_matrix_01 *= 10000  #convert from m2 to cm2

state_ANSI_map_01 = data_fn.regrid001_to_01(state_ANSI_map, Lat_01, Lon_01)

# Print time
ct = datetime.datetime.now() 
print("current time:", ct) 

-------------
## Step 2: Read-in and Format Proxy Data
-------------

#### Step 2.1 Read In Proxy Mapping File & Make Proxy Arrays

In [ ]:
#load GHGI Mapping Groups
names = pd.read_excel(AbdCoal_Mapping_inputfile, sheet_name = "GHGI Map - Abn. Coal", usecols = "A:B",skiprows = 1, header = 0)
colnames = names.columns.values
ghgi_abdcoal_map = pd.read_excel(AbdCoal_Mapping_inputfile, sheet_name = "GHGI Map - Abn. Coal", usecols = "A:B", skiprows = 1, names = colnames)
#drop rows with no data, remove the parentheses and ""
ghgi_abdcoal_map = ghgi_abdcoal_map[ghgi_abdcoal_map['GHGI_Emi_Group'] != 'na']
ghgi_abdcoal_map = ghgi_abdcoal_map[ghgi_abdcoal_map['GHGI_Emi_Group'].notna()]
ghgi_abdcoal_map['GHGI_Source']= ghgi_abdcoal_map['GHGI_Source'].str.replace(r"\(","")
ghgi_abdcoal_map['GHGI_Source']= ghgi_abdcoal_map['GHGI_Source'].str.replace(r"\)","")
ghgi_abdcoal_map.reset_index(inplace=True, drop=True)
display(ghgi_abdcoal_map)

#load emission group - proxy map
names = pd.read_excel(AbdCoal_Mapping_inputfile, sheet_name = "Proxy Map - Abn. Coal", usecols = "A:C",skiprows = 1, header = 0)
colnames = names.columns.values
proxy_abdcoal_map = pd.read_excel(AbdCoal_Mapping_inputfile, sheet_name = "Proxy Map - Abn. Coal", usecols = "A:C", skiprows = 1, names = colnames)
display((proxy_abdcoal_map))

#create empty proxy and emission group arrays (add months for proxy variables that have monthly data)
for igroup in np.arange(0,len(proxy_abdcoal_map)):
    if proxy_abdcoal_map.loc[igroup, 'Grid_Month_Flag'] ==0:
        vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']] = np.zeros([len(Lat_01),len(Lon_01),num_years])
        vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']+'_nongrid'] = np.zeros([num_years])
    else:
        vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']] = np.zeros([len(Lat_01),len(Lon_01),num_years,num_months])
        vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']+'_nongrid'] = np.zeros([num_years,num_months])
        
    vars()[proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([num_years])
    
   # if proxy_abdcoal_map.loc[igroup,'State_Proxy_Group'] != '-':
   #     if proxy_abdcoal_map.loc[igroup,'State_Month_Flag'] == 0:
   #         vars()[proxy_abdcoal_map.loc[igroup,'State_Proxy_Group']] = np.zeros([len(State_ANSI),num_years])
   #         #print('here')
   #     else:
   #         vars()[proxy_abdcoal_map.loc[igroup,'State_Proxy_Group']] = np.zeros([len(State_ANSI),num_years,num_months])
   # else:
   #     continue # do not make state proxy variable if no variable assigned in mapping file
        
emi_group_names = np.unique(ghgi_abdcoal_map['GHGI_Emi_Group'])

print('QA/QC: Is the number of emission groups the same for the proxy and emissions tabs?')
if (len(emi_group_names) == len(np.unique(proxy_abdcoal_map['GHGI_Emi_Group']))):
    print('PASS')
else:
    print('FAIL')
    print(emi_group_names)

#### Step 2.2. Create and Format Mine Level Information

##### Step 2.2.1. Read in Abandoned Mine List

In [ ]:
# Read in Abandoned coal mine list - from the GHGI (this list was added starting with the 1990-2019 GHGI workbook). 
# Future updates to this code should update this file to refelct the latest GHGI workbook. 

abdmines = names = pd.read_excel(EPA_abdcoal_2019_inputfile, sheet_name = "Mine List", usecols = "A:K",skiprows = 5, header = 0)

abdmines = abdmines[abdmines['Year'] != 'x'].copy()
abdmines.reset_index(inplace=True, drop=True)
abdmines.loc[:,'Date of Aban.'] = abdmines.loc[:,'Date of Aban.'].dt.strftime("%m/%d/%y")

list_duplicated = abdmines['MSHA ID'][abdmines['MSHA ID'].duplicated()]
print('Duplicate MSHA IDs: ', np.array(list_duplicated[list_duplicated.notna()]))

display(abdmines)


##### Step 2.2.2. Read in mine locations from MSHA database

In [ ]:
#Read mine locations from MHSA (data.gov) - Same as the one used for coal mining
mine_loc = pd.read_csv(Mine_loc_inputfile,sep="|",encoding= 'unicode_escape')

mine_loc = mine_loc[mine_loc['COAL_METAL_IND']=='C']
#display(mine_loc)
mine_loc = mine_loc[mine_loc['LONGITUDE']!=0]
mine_loc.reset_index(inplace=True, drop=True)
mine_loc['date_abd'] = ''
#display(mine_loc.head(5))

#Put in dates consistent with the other array
for imine in np.arange(len(mine_loc)):
    mine_loc.loc[imine,'date_abd'] = mine_loc['CURRENT_STATUS_DT'][imine][:2] + '/' + mine_loc['CURRENT_STATUS_DT'][imine][3:5] + '/' + mine_loc['CURRENT_STATUS_DT'][imine][8:10]
    if mine_loc.loc[imine,'LONGITUDE'] >0:
        mine_loc.loc[imine,'LONGITUDE'] = -mine_loc.loc[imine,'LONGITUDE'] #logitudes must be set to negative (raw data have mix of + and - longitudes)
# Current mine status from the MSHA database isn't used in the opportunities database so also not used here. 

#mine_loc.head(1)

#### Step 2.2.3. Add location, status, and county to abandoned mine list

In [ ]:
# Add mine locations, status, basin code, and county to abandoned mine list

# Step 1 - Find mine location based on series of checks (based on MSHA ID, state, county, mine name, abandonment date)
# Step 2 - Clean up text on current status and recovery status
# Step 3 - Add basin number
# Step 4 - Format and record the days since abandonment 
# Step 5 - Clean up county name and add to mine array

# Initialize variables
abdmines.loc[:,'lon'] = 0.0
abdmines.loc[:,'lat'] = 0.0
abdmines.loc[:,'Status'] = ''
abdmines.loc[:,'Recovering'] = 0
abdmines.loc[:,'Basin_nr'] = 5 #Initialize to an error value (there are 5 basins)

# Make sure we only use mines once
mine_loc['used'] = 0

mines_matched_msha = []
mines_matched_namedate = []
mines_matched_date = []
mines_matched_name = []

for iyear in np.arange(0,num_years): #Initialize columns for days closed for each year
    col_name = 'Days_closed_'+year_range_str[iyear]
    abdmines.loc[:,col_name]=0.0
    col_name = 'Fraction_year_closed_'+year_range_str[iyear]
    abdmines.loc[:,col_name]=1.0
#print(np.shape(abdmines))

    
for imine in np.arange(0,len(abdmines)):
    ## Step 1 - find mine location
    location_found = 0
    
    # 1A) First, check to see if there is a match based on MSHA ID (between MSHA & Abd mine datasets). If so, record the mine location
    # get rid of dashes in MSHA IDs in the abandoned mines data set
    #print(imine, abdmines['MSHA ID'][imine])
    abdmines['MSHA ID'][imine] = str(abdmines['MSHA ID'][imine])
    try:
        MSHA_String = abdmines['MSHA ID'][imine].replace('-','')
    except:
        MSHA_String = abdmines['MSHA ID'][imine]
        
    matched_id = np.where((mine_loc['MINE_ID'].astype(str) == MSHA_String) & (mine_loc['used'] == 0))[0]
    if len(matched_id) >0:
        matched_id = matched_id[0]
        if mine_loc['LONGITUDE'][matched_id] > Lon_left and mine_loc['LONGITUDE'][matched_id] < Lon_right and \
            mine_loc['LATITUDE'][matched_id] > Lat_low and mine_loc['LATITUDE'][matched_id] < Lat_up:
            abdmines.loc[imine,'lon'] = mine_loc['LONGITUDE'][matched_id]
            abdmines.loc[imine,'lat'] = mine_loc['LATITUDE'][matched_id]
            location_found = 1
            mines_matched_msha.append(imine)
    
    
    # 1B) otherwise, check to see if there is a match of based on state, county, mine name, and abandonment date. If so, record mine location
    matched_mine = np.where((mine_loc['STATE'] == abdmines['State'][imine]) & \
                       (mine_loc['FIPS_CNTY_NM'] == abdmines['County'][imine]) & \
                       (mine_loc['CURRENT_MINE_NAME'] == abdmines['Mine Name'][imine])& \
                       (mine_loc['date_abd'] == abdmines['Date of Aban.'][imine])& (mine_loc['used'] == 0))[0]
    #print(matched_mine)
    if len(matched_mine) > 0 and location_found == 0:
        matched_mine = matched_mine[0]
        if mine_loc['LONGITUDE'][matched_mine] > Lon_left and mine_loc['LONGITUDE'][matched_mine] < Lon_right and \
            mine_loc['LATITUDE'][matched_mine] > Lat_low and mine_loc['LATITUDE'][matched_mine] < Lat_up:
            abdmines.loc[imine,'lon'] = mine_loc['LONGITUDE'][matched_mine]
            abdmines.loc[imine,'lat'] = mine_loc['LATITUDE'][matched_mine]
            location_found = 1
            mines_matched_namedate.append(imine)    
            
    # 1C) otherwise, check to see if there is a match just based on state, county, and date of abandonment (not including mine name)...         
    matched_mine = np.where((mine_loc['STATE'] == abdmines['State'][imine]) & \
                       (mine_loc['FIPS_CNTY_NM'] == abdmines['County'][imine]) & \
                       (mine_loc['date_abd'] == abdmines['Date of Aban.'][imine])& (mine_loc['used'] == 0))[0]
    if len(matched_mine) >0 and location_found == 0:
        matched_mine = matched_mine[0]
        if mine_loc['LONGITUDE'][matched_mine] > Lon_left and mine_loc['LONGITUDE'][matched_mine] < Lon_right and \
            mine_loc['LATITUDE'][matched_mine] > Lat_low and mine_loc['LATITUDE'][matched_mine] < Lat_up:
            abdmines.loc[imine,'lon'] = mine_loc['LONGITUDE'][matched_mine]
            abdmines.loc[imine,'lat'] = mine_loc['LATITUDE'][matched_mine]
            location_found = 1
            mines_matched_date.append(imine)     
            
    # 1D) otherwise, check to see if there is a match based on state, county, and mine name
    #See overrule with name-match
    matched_mine = np.where((mine_loc['STATE'] == abdmines['State'][imine]) & \
                       (mine_loc['FIPS_CNTY_NM'] == abdmines['County'][imine]) & \
                       (mine_loc['CURRENT_MINE_NAME'] == abdmines['Mine Name'][imine])& \
                       (mine_loc['used'] == 0))[0]
    if len(matched_mine) > 0 and location_found == 0:
        matched_mine = matched_mine[0]
        if mine_loc['LONGITUDE'][matched_mine] > Lon_left and mine_loc['LONGITUDE'][matched_mine] < Lon_right and \
        mine_loc['LATITUDE'][matched_mine] > Lat_low and mine_loc['LATITUDE'][matched_mine] < Lat_up:
            abdmines.loc[imine,'lon'] = mine_loc['LONGITUDE'][matched_mine]
            abdmines.loc[imine,'lat'] = mine_loc['LATITUDE'][matched_mine]
            location_found = 1
            mines_matched_name.append(imine)     

    
    ## Step 2. Clean up Current Status and Recovery Status Text
    #Strip leading/trailing
    try:
        abdmines.loc[imine,'Current Emissions Status'] = abdmines['Current Emissions Status'][imine].strip()
    except:
        0
    
    # 2A) Clean up the current status text
    if abdmines['Current Emissions Status'][imine] == 'Flooding' or \
        abdmines['Current Emissions Status'][imine] == 'Flooded/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Recovering Methane/Partially Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Flooded/Pumping Water' or \
        abdmines['Current Emissions Status'][imine] == 'Venting/Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Venting/Partially Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Flooding/Recovering Methane':
        abdmines.loc[imine,'Status'] = 'Flooded'
    elif abdmines['Current Emissions Status'][imine] == 'Sealed Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Partially Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Abandoned/Sealed' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Pumping Water' or \
        abdmines['Current Emissions Status'][imine] == 'Temporary Seal' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Filled' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Recovery Pending' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed*' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Adj to Dianne Mine':
        abdmines.loc[imine,'Status'] = 'Sealed'
    elif abdmines['Current Emissions Status'][imine] == 'Venting/Recovering Gas' or \
        abdmines['Current Emissions Status'][imine] == 'Venting/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Venting':
        abdmines.loc[imine,'Status'] = 'Venting'
    else:
        abdmines.loc[imine,'Status'] = 'Unknown'
        
    #2B) Clean up recovery status text
    if abdmines['Current Emissions Status'][imine] == 'Sealed Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Flooded/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Venting/Recovering Gas' or \
        abdmines['Current Emissions Status'][imine] == 'Recovering Methane/Partially Flooded' or \
        abdmines['Current Emissions Status'][imine] == 'Venting/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Sealed/Recovering Methane' or \
        abdmines['Current Emissions Status'][imine] == 'Flooding/Recovering Methane':
        abdmines.loc[imine,'Recovering'] = 1       

    ## Step 3 - Add basin number. #CA, IL, NA, BW, WS
    # There are 5 basins, Central Appalachia, Illinois, Northern Appalachia, Warrior Basin, and Western Basins
    # Western basins include: Arkoma [OK], Uinta [UT], Piceance [CO], Raton [CO/NM], WTB [CO]
    if abdmines['Coal Basin'][imine] == 'Central Appl.':
        abdmines.loc[imine,'Basin_nr'] = 0
    elif abdmines['Coal Basin'][imine] == 'Illinois':
        abdmines.loc[imine,'Basin_nr'] = 1
    elif abdmines['Coal Basin'][imine] == 'Northern Appl.':
        abdmines.loc[imine,'Basin_nr'] = 2    
    elif abdmines['Coal Basin'][imine] == 'Warrior':
        abdmines.loc[imine,'Basin_nr'] = 3
    else:
        abdmines.loc[imine,'Basin_nr'] = 4

    
    
    ## Step 4. Record the number of days since the mine has closed
    #Add number of days since closure
    for iyear in np.arange(0,num_years):
        if year_range[iyear]==2012 or year_range[iyear]==2016:
            year_days = np.sum(month_day_leap)
            month_days = month_day_leap
        else:
            year_days = np.sum(month_day_nonleap)
            month_days = month_day_nonleap
            
        ab_mon = float(abdmines['Date of Aban.'][imine][:2])
        ab_day = float(abdmines['Date of Aban.'][imine][3:5])
        ab_yea = float(abdmines['Date of Aban.'][imine][6:8])

        #Deal with faulty dates
        if ab_mon > 12:
            ab_day = float(abdmines['Date of Aban.'][imine][:2])
            ab_mon = float(abdmines['Date of Aban.'][imine][3:5])
        
        # Count days relative to 07/02, middle of the year, for flux estimates
        days = 0
        days += 2 - ab_day
        days += 31*(7-ab_mon)

        if ab_yea > 19: # years in the 1900s
            days += year_days*(year_range[iyear] - (1900 + ab_yea))
        else: # years in the 2000s
            days += year_days*(year_range[iyear] - (2000 + ab_yea))

        abdmines.loc[imine,'Days_closed_'+year_range_str[iyear]] = days
        
        #Special treatment for mines that closed the same year
        if year_range[iyear] == (2000 + ab_yea):
            # Calculate average number of days closed by dividing days closed by 2
            days = 0
            days += 31 - ab_day
            days += 30.5*(12-ab_mon)
            abdmines.loc[imine,'Days_closed_'+year_range_str[iyear]] = days/2
            abdmines.loc[imine,'Fraction_year_closed_'+year_range_str[iyear]] = days/year_days
            
## Step 5. Clean up county name and record county and state ANSI numbers in array
abdmines['state'] = 0
abdmines['county'] = 0

for imine in np.arange(0,len(abdmines)):
    abdmines.loc[imine,'state'] = abbr_dict[abdmines['State'][imine].strip()]
    #Fix typos
    if abdmines['County'][imine] == 'Clearborne':
        abdmines.loc[imine,'County'] = 'claiborne'
    if abdmines['County'][imine] == 'Dickerson':
        abdmines.loc[imine,'County'] = 'dickenson'
    if abdmines['County'][imine] == 'Davies':
        abdmines.loc[imine,'County'] = 'daviess'
    if abdmines['County'][imine] == 'Rosedale':
        abdmines.loc[imine,'County'] = 'trousdale'          
         
    match_arr = np.where((County_ANSI['State']==abdmines['state'][imine]) & \
                         (County_ANSI['Name']==abdmines['County'][imine].strip().lower()))[0]
    if len(match_arr) > 0:
        match_arr = match_arr[0]
        abdmines.loc[imine,'county'] = County_ANSI['County'][match_arr]
    else:
        abdmines.loc[imine,'county'] = 0

#display(abdmines.head(1))

# Report the number of mines matched based on each method. 
print('Only matched if there is a unique match')
print('Matched on MSHA:      ', len(mines_matched_msha))
print('Matched on name+date: ', len(mines_matched_namedate))
print('Matched on date:      ', len(mines_matched_date))
print('Matched on name:      ', len(mines_matched_name))

#Throw out mines without either county or latitude [This throws out 2 mine]
abdmines = abdmines[(abdmines['county'] > 0)|(abdmines['lat'] > 0)]
abdmines.reset_index(inplace=True, drop=True)

##### Step 2.2.4 Check for Mines that have been reopened

In [ ]:
# Check abandoned mines database for mines that have reopened

#NOTES:
# 1) Mines reopened in 2020 will not affect this notebook run for 2012-2018. 
# 2) MSHA says mine 3600840 is active, but it is not in active mine GHGI workbook. It is in the abandoned mine workbook.
#    abandoned in 1994. We keep it here. 
# 3) Mine 4200079 is present in active mines notebook and also has abandoned emissions. However, 
#    its emissions are only ~0.8% of Utah's abandoned mines emissions. We keep it here. 
#    Old: We remove it in the flux calculation block for 2016-2018. 
# 4) Mine 1100588 is listed as a refuse recovery mine in the active mines notebook, with coal production for the
#    years 2012-2016. Refuse recovery mine emissions are not included in the active mining emissions estimates. 
#    This mine also has abandoned emissions and is in the abandoned GHGI workbook (close in 1995). We keep this here.
#    #old: We remove it in the flux calculation block here for years 2014-2018.
# 5) Check to see if there are other mines listed below. 
checker = abdmines['MSHA ID']
checker[checker == 'nan'] = np.nan
checker.dropna(inplace=True)
check = checker.reset_index()
print('REOPENED MINES')
for idi in np.arange(len(check)):
    ind = check['MSHA ID'].iloc[idi]
    temp = mine_loc[mine_loc['MINE_ID']==int(ind.replace("-", ""))]
    #print(temp)
    if len(temp)>0:
        if temp['CURRENT_MINE_STATUS'].iloc[0]=='Active':
            print('MSHA ID: ',temp['MINE_ID'].iloc[0])
            print('Date reopened: ', mine_loc[mine_loc['MINE_ID']==int(ind.replace("-", ""))]['CURRENT_CONTROLLER_BEGIN_DT'].iloc[0])
            print('index',np.where(abdmines['MSHA ID']==ind)[0][0])
            print('')
            

#### Step 2.2.5 Calculate Mine-Level Emissions

In [ ]:
#Calculate the timeseries of emissions from each mine based on the mine status, time since closure, and the emission 
# decay curves used in the GHGI. 

# Basin specific coefficients for sealed, flooded, and vented emission decay curves
#Hard-coded coefficients, we use medium values for everything
#Old order: NA, IL, CA, BW, WS
#New order: CA, IL, NA, WB, WS
b_medium = np.array([2.329011,2.314585,2.292595,2.299685,2.342465])
D_sealed  = np.array([0.000741,0.000733,0.000725,0.000729,0.000747]) 
D_venting  = np.array([0.003735,0.003659,0.003564,0.003601,0.003803])
D_flooded = np.array([0.672,0.672,0.672,0.672,0.672])

# Calculate emissions based on years since closure and the mine status (if known)
# If the mine status is unknown, emissions are estimated based on the fraction of mines in the given basin that are
# flooded, venting or sealed. 
# These basin-specific mine type fractions are read in from the GHGI workbook ([year] tabs)

# Initialize flux column for each year
for iyear in np.arange(0, num_years):
    #abdmines['Flux_'+str(year_list[iyear])] = 0.0
    abdmines['Emis_mmcfd_'+year_range_str[iyear]] = 0.0
    abdmines['Emis_tg_'+year_range_str[iyear]] = 0.0
    
    
for iyear in np.arange(0, num_years):
    if year_range[iyear]==2012 or year_range[iyear]==2016:
        year_days = np.sum(month_day_leap)
    else:
        year_days = np.sum(month_day_nonleap)
        
    if year_range[iyear] == 2012 or year_range[iyear] == 2018:
        start_row = 26
    else:
        start_row = 29
        
    #Read the fraction of mine types for the given year
    Ratios = pd.read_excel(EPA_abdcoal_inputfile,year_range_str[iyear],skiprows=start_row,nrows=5,usecols=np.arange(4))

    fra_sealed  = np.array(Ratios['Sealed %'])
    fra_venting = np.array(Ratios['Vented %'])
    fra_flooded = np.array(Ratios['Flooded %'])

    #Normalize to 100%
    fra_summed = fra_sealed+fra_venting+fra_flooded
    fra_sealed  = fra_sealed  / fra_summed
    fra_venting = fra_venting / fra_summed
    fra_flooded = fra_flooded / fra_summed
    mine_count = 0
    
    for imine in np.arange(0,len(abdmines)):
        ibasin = abdmines['Basin_nr'][imine]
        ab_numyrs = abdmines['Days_closed_'+year_range_str[iyear]][imine]/year_days
        ab_yea = float(abdmines['Date of Aban.'][imine][6:8])

        if ab_numyrs >= 0:
            if abdmines['Active Emiss. (mmcfd) '][imine] > 0:
                #Deal with recovery projects
                if abdmines['Recovering'][imine] == 1:
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = 0.0
                elif abdmines['Status'][imine] == 'Flooded':
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = abdmines['Active Emiss. (mmcfd) '][imine] * \
                        np.exp(-1*D_flooded[ibasin]*ab_numyrs)
                elif abdmines['Status'][imine] == 'Venting':
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = abdmines['Active Emiss. (mmcfd) '][imine] * \
                        (1+b_medium[ibasin]*D_venting[ibasin]*ab_numyrs)**(-1/float(b_medium[ibasin]))
                elif abdmines['Status'][imine] == 'Sealed':
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = abdmines['Active Emiss. (mmcfd) '][imine] * \
                    (1-0.8) * (1+b_medium[ibasin]*D_sealed[ibasin]*ab_numyrs)**(-1/float(b_medium[ibasin]))
                else:
                    em_flo = abdmines.loc[imine,'Active Emiss. (mmcfd) '] * np.exp(-1*D_flooded[ibasin]*ab_numyrs)
                    em_ven = abdmines.loc[imine,'Active Emiss. (mmcfd) '] * (1+b_medium[ibasin]*D_venting[ibasin]*ab_numyrs)**(-1/float(b_medium[ibasin]))
                    em_sea = abdmines.loc[imine,'Active Emiss. (mmcfd) '] * (1-0.8) * (1+b_medium[ibasin]*D_sealed[ibasin]*ab_numyrs)**(-1/float(b_medium[ibasin]))
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = fra_flooded[ibasin]*em_flo + fra_venting[ibasin]*em_ven + fra_sealed[ibasin]*em_sea
                    
                #Special treatment for mines that closed the same year
                # Given that we are reporting annual emissions, the emissions from mines closed in the given year
                # are scaled down to reflect that they were not emitting at the above calculated rate for the entire
                # year, but only a fraction of the year
                # annual emissions rate  = calculated flow rate * fraction of year closed
                if year_range[iyear] == (2000 + ab_yea):
                    mine_count +=1
                    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = \
                        abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]]*abdmines.loc[imine,'Fraction_year_closed_'+year_range_str[iyear]]
            
        #Remove active mine years [See block above]
        #if abdmines['MSHA ID'][imine] == '4200079' and year_range[iyear] > 2015:
        #    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = np.nan
        #if abdmines['MSHA ID'][imine] == '1100588' and year_range[iyear] > 2014:
        #    abdmines.loc[imine,'Emis_mmcfd_'+year_range_str[iyear]] = np.nan
    
    #Report number of mines that were closed during the current year
    print('Mines closed in the current year:',mine_count,'of',len(abdmines))
    # Set emission value by scaling against national total
    #abdmines['Emis_tg_'+year_range_str[iyear]] = abdmines['Emis_mmcfd_'+year_range_str[iyear]]/float(np.sum(abdmines['Emis_mmcfd_'+year_range_str[iyear]]))*EPA_emi_abdcoal_total.iloc[0,iyear+1]  
    
    
# Replace NaN with zero 
for iyear in np.arange(0, num_years):
    abdmines['Emis_mmcfd_'+year_range_str[iyear]] = abdmines['Emis_mmcfd_'+year_range_str[iyear]].fillna(0)

#### Step 2.3. Place Mine-Level Emissions onto a 0.01x0.01 CONUS Grid

In [ ]:
#Allocate emissions to a 0.01x0.01 degree CONUS grid
# Note that all emissions are from basins within the CONUS region, and therefore there are no 'non-grid' emissions

map_mine_emis = np.zeros([len(lat001),len(lon001),num_years])
map_mine_emis_nongrid = np.zeros([num_years])
CM_Emissions = np.zeros([len(State_ANSI),len(County_ANSI),num_years])

print('QA/QC: Number of missed mine locations (emissions allocated to the county-level)')
for iyear in np.arange(0, num_years):
    missed_locations = 0 
    missed_emis = 0
    for imine in np.arange(0,len(abdmines)):
        ansi_state = abdmines['state'][imine]
        ansi_county = abdmines['county'][imine]
        
        if abdmines['lat'][imine] > 0:
            #Set ilon and ilat
            ilat = int((abdmines['lat'][imine] - Lat_low)/Res_01)
            ilon = int((abdmines['lon'][imine] - Lon_left)/Res_01)
            
        #Check is given location matches with state [Or if match is one grid box to either side cause we may be at the edges of states]
            if np.sum(state_ANSI_map[ilat-1:ilat+2,ilon-1:ilon+2] == abdmines['state'][imine]) > 0:
                map_mine_emis[ilat,ilon,iyear] += abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]# * 10**12 * Avagrado / float(Molarch4 * 366 * 24 * 60 * 60) / float(10000*are_map[i_lat,i_lon])
            else:
                # If the state isn't correct, allocate based on the county level because the lat/lon is probably way off
                missed_locations += 1
                CM_Emissions[ansi_state,ansi_county,iyear] += abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]
                mask_county = np.ma.ones(np.shape(county_ANSI_map))
                mask_county = np.ma.masked_where(county_ANSI_map != ansi_county, mask_county)
                mask_county = np.ma.masked_where(state_ANSI_map != ansi_state, mask_county)
                mask_county = np.ma.filled(mask_county,0)
                # allocate the emissions evenly across all grid cells in county (e.g., mine emissions * area in grid cell in county/total county area)
                map_mine_emis[:,:,iyear]+=((mask_county * area_map)/np.sum(mask_county * area_map))*abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]
                missed_emis += abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]
        else:
            missed_locations +=1
            CM_Emissions[ansi_state,ansi_county,iyear] += abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]
            mask_county = np.ma.ones(np.shape(county_ANSI_map))
            mask_county = np.ma.masked_where(county_ANSI_map != ansi_county, mask_county)
            mask_county = np.ma.masked_where(state_ANSI_map != ansi_state, mask_county)
            mask_county = np.ma.filled(mask_county,0)
            # allocate the emissions evenly across all grid cells in county (e.g., mine emissions * area in grid cell in county/total county area)
            map_mine_emis[:,:,iyear]+=((mask_county * area_map)/np.sum(mask_county * area_map))*abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]
            missed_emis += abdmines['Emis_mmcfd_'+year_range_str[iyear]][imine]

    print('Year', year_range[iyear],':', missed_locations)
    print('Percent of emissions allocated to the county level:', missed_emis/np.sum(map_mine_emis[:,:,iyear]))

-----------
## Step 3. Read in and Format US EPA GHGI Emissions
----------

In [ ]:
#Read in Data from InvDB tab in the Inventory workbook (Tg == 1000 kt)

names = pd.read_excel(EPA_abdcoal_inputfile, sheet_name = "InvDB", usecols = "A:AJ",skiprows = 15, header = 0)
colnames = names.columns.values
EPA_emi_abdcoal_CH4 = pd.read_excel(EPA_abdcoal_inputfile, sheet_name = "InvDB", usecols = "A:AJ", skiprows = 15, nrows = 2,names = colnames)
EPA_emi_abdcoal_CH4 = EPA_emi_abdcoal_CH4.fillna('')
EPA_emi_abdcoal_CH4 = EPA_emi_abdcoal_CH4.drop(columns = [n for n in range(1990, start_year,1)])
EPA_emi_abdcoal_CH4 = EPA_emi_abdcoal_CH4.drop(columns = ['Sector','Source','State','Subsource','Fuel','GHG'])
EPA_emi_abdcoal_CH4.rename(columns={'Subref':'Source'},inplace=True)
EPA_emi_abdcoal_CH4.reset_index(inplace=True, drop=True)

#calculate national total from state values
temp = EPA_emi_abdcoal_CH4.sum(axis=0)
EPA_emi_abdcoal_CH4 = EPA_emi_abdcoal_CH4.append(temp, ignore_index=True)
EPA_emi_abdcoal_CH4.iloc[-1,0] = 'Total'
EPA_emi_abdcoal_total = EPA_emi_abdcoal_CH4[EPA_emi_abdcoal_CH4['Source'] == 'Total']

display(EPA_emi_abdcoal_total)

#### 3.2. Split Emissions into Gridding Groups

In [ ]:
#split GHG emissions into gridding groups, based on Coal Proxy Mapping file

DEBUG =1
start_year_idx = EPA_emi_abdcoal_CH4.columns.get_loc((start_year))
end_year_idx = EPA_emi_abdcoal_CH4.columns.get_loc((end_year))+1
ghgi_abdcoal_groups = ghgi_abdcoal_map['GHGI_Emi_Group'].unique()
sum_emi = np.zeros([num_years])

for igroup in np.arange(0,len(ghgi_abdcoal_groups)): #loop through all groups, finding the GHGI sources in that group and summing emissions for that region, year        vars()[ghgi_prod_groups[igroup]] = np.zeros([num_regions-1,num_years])
    ##DEBUG## print(ghgi_stat_groups[igroup])
    vars()[ghgi_abdcoal_groups[igroup]] = np.zeros([num_years])
    source_temp = ghgi_abdcoal_map.loc[ghgi_abdcoal_map['GHGI_Emi_Group'] == ghgi_abdcoal_groups[igroup], 'GHGI_Source']
    pattern_temp  = '|'.join(source_temp) 
    emi_temp =EPA_emi_abdcoal_CH4[EPA_emi_abdcoal_CH4['Source'].str.contains(pattern_temp)]
    vars()[ghgi_abdcoal_groups[igroup]][:] = emi_temp.iloc[:,start_year_idx:].sum()
        
        
#Check against total summary emissions 
print('QA/QC #1: Check Processing Emission Sum against GHGI Summary Emissions')
for iyear in np.arange(0,num_years): 
    for igroup in np.arange(0,len(ghgi_abdcoal_groups)):
        sum_emi[iyear] += vars()[ghgi_abdcoal_groups[igroup]][iyear]
        
    summary_emi = EPA_emi_abdcoal_total.iloc[0,iyear+1]  
    #Check 1 - make sure that the sums from all the regions equal the totals reported
    diff1 = abs(sum_emi[iyear] - summary_emi)/((sum_emi[iyear] + summary_emi)/2)
    if DEBUG==1:
        print(summary_emi)
        print(sum_emi[iyear])
    if diff1 < 0.0001:
        print('Year ', year_range[iyear],': PASS, difference < 0.01%')
    else:
        print('Year ', year_range[iyear],': FAIL (check Production & summary tabs): ', diff1,'%')

--------------
## Step 4. Grid Data
-------------

#### Step 4.1. Allocate emissions

##### Step 4.1.1 Assign the Appropriate Proxy Variable Names (state & grid)

In [ ]:
# The names on the *left* need to match the 'AbandonedCoal_ProxyMapping' names 
# (these are initialized in Step 2). 
# The names on the *right* are the variable names used to caluclate the proxies in this code.
# Names on the right need to match those from the code in Step 2

#national --> grid proxies (0.01x0.01, year)
Map_AbdCoal = map_mine_emis
Map_AbdCoal_nongrid = map_mine_emis_nongrid


##### Step 4.1.2 Allocate emissions to the CONUS region (0.1x0.1)

In [ ]:
# Allocate national emissions (Tg) onto a 0.1x0.1 grid using gridcell level 'Proxy_Groups'

DEBUG =1
#Define emission arrays
Emissions_array_01 = np.zeros([len(Lat_01),len(Lon_01),num_years])
Emissions_array_001 = np.zeros([len(lat001),len(lon001),num_years])
Emissions_nongrid = np.zeros([num_years])

# For each year, distribute natinal emissions onto a grid proxies specified in the Proxy_Mapping file

print('**QA/QC Check: Sum of national gridded emissions vs. GHGI national emissions')
for igroup in np.arange(len(proxy_abdcoal_map)):
    vars()['Ext_'+proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']] = np.zeros([len(Lat_01),len(Lon_01),num_years])


for igroup in np.arange(0,len(proxy_abdcoal_map)):
    proxy_temp = vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']]
    proxy_temp_nongrid = vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']+'_nongrid']

    for iyear in np.arange(0,num_years):
        temp_sum = np.sum(vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']][:,:,iyear])+np.sum(vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']+'_nongrid'][iyear])
        emi_temp = vars()[proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']][iyear] * \
                       data_fn.safe_div(vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']][:,:,iyear], temp_sum)
        Emissions_array_001[:,:,iyear] += emi_temp
        emi_temp_01 = data_fn.regrid001_to_01(emi_temp, Lat_01, Lon_01)
        vars()['Ext_'+proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']][:,:,iyear] = emi_temp_01
        Emissions_nongrid[iyear] += vars()[proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']][iyear] *\
                        data_fn.safe_div(vars()[proxy_abdcoal_map.loc[igroup,'Proxy_Group']+'_nongrid'][iyear], temp_sum)

for iyear in np.arange(0, num_years):    
    Emissions_array_01[:,:,iyear] = data_fn.regrid001_to_01(Emissions_array_001[:,:,iyear], Lat_01, Lon_01)
    calc_emi = 0
    for igroup in np.arange(0,len(proxy_abdcoal_map)):
        calc_emi += np.sum(vars()['Ext_'+proxy_abdcoal_map.loc[igroup,'GHGI_Emi_Group']][:,:,iyear])
    calc_emi += np.sum(Emissions_nongrid[iyear]) 
    #calc_emi = np.sum(Emissions_array_01[:,:,iyear]) + np.sum(Emissions_nongrid[iyear]) 
    summary_emi = EPA_emi_abdcoal_total.iloc[0,iyear+1]
    emi_diff = abs(summary_emi-calc_emi)/((summary_emi+calc_emi)/2)
    if DEBUG==1:
        print(calc_emi)
        print(summary_emi)
    if abs(emi_diff) < 0.0001:
        print('Year '+ year_range_str[iyear]+': Difference < 0.01%: PASS')
    else: 
        print('Year '+ year_range_str[iyear]+': Difference > 0.01%: FAIL, diff: '+str(emi_diff))
        
ct = datetime.datetime.now() 
print("current time:", ct)

#### Step 4.1.4 Save gridded emissions (kt)

In [ ]:
#save gridded emissions for each gridding group - for extension

#Initialize file
data_IO_fn.initialize_netCDF(grid_emi_outputfile, netCDF_description, 0, year_range, loc_dimensions, Lat_01, Lon_01)

unique_groups = np.unique(proxy_abdcoal_map['GHGI_Emi_Group'])
unique_groups = unique_groups[unique_groups != 'Emi_not_mapped']

nc_out = Dataset(grid_emi_outputfile, 'r+', format='NETCDF4')

for igroup in np.arange(0,len(unique_groups)):
    print('Ext_'+unique_groups[igroup])
    if len(np.shape(vars()['Ext_'+unique_groups[igroup]])) ==4:
        ghgi_temp = np.sum(vars()[unique_groups[igroup]],axis=3) #sum month data if data is monthly
    else:
        ghgi_temp = vars()['Ext_'+unique_groups[igroup]]

    # Write data to netCDF
    data_out = nc_out.createVariable('Ext_'+unique_groups[igroup], 'f8', ('lat', 'lon','year'), zlib=True)
    data_out[:,:,:] = ghgi_temp[:,:,:]

#save nongrid data to calculate non-grid fraction extension
data_out = nc_out.createVariable('Emissions_nongrid', 'f8', ('year'), zlib=True)  
data_out[:] = Emissions_nongrid[:]
nc_out.close()

#Confirm file location
print('** SUCCESS **')
print("Gridded emissions (kt) written to file: {}" .format(os.getcwd())+grid_emi_outputfile)
print(' ')

del data_out, ghgi_temp, nc_out

#### 4.2 Calculate Gridded Emission Fluxes (molec./cm2/s) (0.1x0.1)

In [ ]:
#Convert emissions to emission flux
# conversion: Tg emissions to molec/cm2/s flux

Flux_array_01_annual = np.zeros([len(Lat_01),len(Lon_01),num_years])
print('**QA/QC Check: Sum of national gridded emissions vs. GHGI national emissions')
  
for iyear in np.arange(0,num_years):
    calc_emi = 0
    if year_range[iyear]==2012 or year_range[iyear]==2016:
        year_days = np.sum(month_day_leap)
    else:
        year_days = np.sum(month_day_nonleap)

    conversion_factor_01 = 10**12 * Avogadro / float(Molarch4 *year_days * 24 * 60 *60) / area_matrix_01
    Flux_array_01_annual[:,:,iyear] = Emissions_array_01[:,:,iyear]*conversion_factor_01
    #convert back to mass to check
    conversion_factor_annual = 10**12 * Avogadro / float(Molarch4 *year_days * 24 * 60 *60) / area_matrix_01
    calc_emi = np.sum(Flux_array_01_annual[:,:,iyear]/conversion_factor_annual)+np.sum(Emissions_nongrid[iyear])
    summary_emi = EPA_emi_abdcoal_total.iloc[0,iyear+1]
    emi_diff = abs(summary_emi-calc_emi)/((summary_emi+calc_emi)/2)
    if DEBUG==1:
        print(calc_emi)
        print(summary_emi)
    if abs(emi_diff) < 0.0001:
        print('Year '+ year_range_str[iyear]+': Difference < 0.01%: PASS')
    else: 
        print('Year '+ year_range_str[iyear]+': Difference > 0.01%: FAIL, diff: '+str(emi_diff))
        
Flux_Emissions_Total_annual = Flux_array_01_annual

-------------
## Step 5. Write netCDF
------------

In [ ]:
# yearly data
#Initialize file
data_IO_fn.initialize_netCDF(gridded_outputfile, netCDF_description, 0, year_range, loc_dimensions, Lat_01, Lon_01)

# Write data to netCDF
nc_out = Dataset(gridded_outputfile, 'r+', format='NETCDF4')
nc_out.variables['emi_ch4'][:,:,:] = Flux_Emissions_Total_annual
nc_out.close()
#Confirm file location
print('** SUCCESS **')
print("Gridded abandoned underground mine fluxes written to file: {}" .format(os.getcwd())+gridded_outputfile)

----------
## Step 6. Plot Gridded Data
---------

#### Step 6.1. Plot Annual Emission Fluxes

In [ ]:
#Plot Annual Data
scale_max = 10
save_flag = 0
save_outfile = ''
data_plot_fn.plot_annual_emission_flux_map(Flux_Emissions_Total_annual, Lat_01, Lon_01, year_range, title_str,scale_max,save_flag,save_outfile)

#### Step 6.2 Plot Difference between first and last inventory year

In [ ]:
# Plot difference between last and first year
save_flag = 0
save_outfile = ''
data_plot_fn.plot_diff_emission_flux_map(Flux_Emissions_Total_annual, Lat_01, Lon_01, year_range, title_diff_str,save_flag,save_outfile)

In [ ]:
ct = datetime.datetime.now() 
ft = ct.timestamp() 
time_elapsed = (ft-it)/(60*60)
print('Time to run: '+str(time_elapsed)+' hours')
print('** GEPA_1B1a_Coal_Abandoned: COMPLETE **')